In [154]:
import warnings

warnings.filterwarnings("ignore")

import os
import shutil
import platform
import cv2
import pandas as pd
import random
import math
import uuid
import sys
import traceback
import utils

%load_ext autoreload
%autoreload 2

%reload_ext nb_black
%config IPCompleter.greedy=True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [125]:
CURRENT_MICE = "PrL-16"
parent_directory = os.path.dirname(os.getcwd())
path_to_videos = os.path.join(parent_directory, "videos")
path_to_data = os.path.join(parent_directory, "data")

mice = [
    "PrL-2",
    "PrL-3",
    "PrL-4",
    "PrL-5",
    "PrL-8",
    "PrL-9",
    "PrL-10",
    "PrL-16",
    "PrL-19",
]
labels = ["parent-mice", "child-mice"]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [126]:
mice_df = pd.DataFrame(mice, columns=["Mice"])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [127]:
def get_video_data(mice: str):
    video_file_path = f"{path_to_videos}/{mice}.mp4"

    if not os.path.isfile(video_file_path):
        raise FileNotFoundError(f"{mice}.mp4 does not exist!")

    videoCaptureInstance = cv2.VideoCapture(video_file_path)
    fps = videoCaptureInstance.get(cv2.CAP_PROP_FPS)
    frames = videoCaptureInstance.get(cv2.CAP_PROP_FRAME_COUNT)
    return pd.Series([fps, frames])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [128]:
mice_df[["video_fps", "total_video_frames"]] = mice_df["Mice"].apply(
    lambda mice: get_video_data(mice)
)
mice_df.head()

,Mice,video_fps,total_video_frames
0,PrL-2,30.00003,83713.0
1,PrL-3,30.00003,40655.0
2,PrL-4,30.00003,48634.0
3,PrL-5,30.00003,46981.0
4,PrL-8,30.00003,48489.0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [129]:
mice_row = mice_df[mice_df.Mice == CURRENT_MICE]
mice_row

,Mice,video_fps,total_video_frames
7,PrL-16,30.00003,41131.0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [130]:
def create_images_from_video(start_seconds: int, end_seconds: int, class_label: str):

    if class_label not in labels:
        raise ValueError("Enter a valid class label!")
    if CURRENT_MICE not in mice:
        raise ValueError("Enter a valid mice!")

    if not os.path.isdir(f"{path_to_data}/{class_label}"):
        os.mkdir(f"{path_to_data}/{class_label}")

    frame_rate = math.floor(mice_row["video_fps"].item())

    video_file_path = f"{path_to_videos}/{CURRENT_MICE}.mp4"
    videoCaptureInstance = cv2.VideoCapture(video_file_path)

    start_frame = start_seconds * frame_rate
    seconds = start_seconds
    try:
        while seconds != end_seconds:
            ret, frame = videoCaptureInstance.read()
            if not ret:
                raise BufferError(
                    "Unexpected error occurred with reading image due to bad video file!"
                )
                break

            # Record frame every 1 second
            if start_frame % frame_rate == 0:
                filename = f"{CURRENT_MICE}-{uuid.uuid4()}"
                filepath = f"{path_to_data}/{class_label}/{filename}.jpg"
                cv2.imwrite(filepath, frame)

                print(
                    f"Saved image as {filename} at timestamp {utils.sec_to_time(seconds)}"
                )
                seconds += 1
            client_key_press = cv2.waitKey(1) & 0xFF

            # end video stream by escape key
            if client_key_press == 27:
                break
            start_frame += 1

        videoCaptureInstance.release()

        if platform.system() == "Darwin":
            cv2.waitKey(1)

    except cv2.error:
        print(
            "Unexpected error occurred with reading image and error code is",
            cv2.error.code,
        )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [133]:
def split_train_test(class_label: str, split_ratio: float):

    if class_label not in labels:
        raise ValueError("Enter a valid class label!")

    split_ratio = split_ratio
    all_mice_files = os.listdir(f"{path_to_data}/{class_label}")
    random.shuffle(all_mice_files)
    train_size = len(all_mice_files) * split_ratio
    files_moved = 0

    for mice in all_mice_files:
        if files_moved < train_size:
            shutil.move(
                f"{path_to_data}/{class_label}/{mice}",
                f"{path_to_data}/train/{class_label}",
            )
        else:
            shutil.move(
                f"{path_to_data}/{class_label}/{mice}",
                f"{path_to_data}/test/{class_label}",
            )
        files_moved += 1
    if len(os.listdir(f"{path_to_data}/{class_label}")) == 0:
        shutil.rmtree(f"{path_to_data}/{class_label}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [148]:
def create_dataset_pipeline(
    start_seconds, end_seconds, class_label: str, split=True, split_ratio=0.8
):
    try:
        create_images_from_video(start_seconds, end_seconds, class_label)
        if split:
            if len(os.listdir(f"{path_to_data}/{class_label}")) != 0:
                split_train_test(class_label, split_ratio)
            else:
                sys.exit(
                    f"{path_to_data}/{class_label} is empty and that means frames from videos were not captured successfully!"
                )
        else:
            pass
    except:
        var = traceback.format_exc()
        print(var)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [149]:
# start = floor(Total frames / frame_rate) - (ignore last trial(120 seconds) + 30 second threshold)
def create_parent_dataset(parent_images):
    frame_rate = math.floor(mice_row["video_fps"].item())
    start_seconds = random.randint(
        0, (mice_row["total_video_frames"].item() // frame_rate) - 150
    )
    end_seconds = start_seconds + parent_images
    create_dataset_pipeline(start_seconds, end_seconds, "parent-mice")

    train_size = len(os.listdir(f"{path_to_data}/train/parent-mice"))
    test_size = len(os.listdir(f"{path_to_data}/test/parent-mice"))

    print(
        f"Saved all {parent_images} parent image files and train has {train_size} pictures and test has {test_size} pictures!"
    )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [136]:
experiment = pd.read_csv(f"{parent_directory}/experiment_new.csv")
experiment.head()

,Group,Cohort,MouseID,Virus,Trial,Laser,Start,Time_Latency_Contact_s,Latency_Contact_s,Time_Latency_Retrieve_s,Latency_Retrieve_s,Retrieved
0,Control,1,PrL-10,mCherry,1,OFF,0:13,0:27,14.0,NO,120,0
1,Control,1,PrL-10,mCherry,2,ON,2:38,2:57,19.0,NO,120,0
2,Control,1,PrL-10,mCherry,3,OFF,5:00,6:11,71.0,NO,120,0
3,Control,1,PrL-10,mCherry,4,ON,7:19,8:27,68.0,NO,120,0
4,Control,1,PrL-10,mCherry,5,OFF,9:37,NO,120.0,NO,120,0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [137]:
curr_mouse_df = experiment[
    (experiment.MouseID == CURRENT_MICE)
    & (experiment.Time_Latency_Contact_s != "NO")
    & (experiment.Time_Latency_Retrieve_s != "NO")
]
curr_mouse_df

,Group,Cohort,MouseID,Virus,Trial,Laser,Start,Time_Latency_Contact_s,Latency_Contact_s,Time_Latency_Retrieve_s,Latency_Retrieve_s,Retrieved
67,Control,2,PrL-16,mCherry,2,ON,2:18,2:40,22.0,2:48,30,1
68,Control,2,PrL-16,mCherry,3,OFF,4:33,4:40,7.0,4:45,12,1
69,Control,2,PrL-16,mCherry,4,ON,6:41,6:54,13.0,6:59,18,1
70,Control,2,PrL-16,mCherry,5,OFF,9:06,9:24,18.0,9:29,13,1
71,Control,2,PrL-16,mCherry,6,ON,11:15,12:00,45.0,12:08,53,1
73,Control,2,PrL-16,mCherry,8,ON,15:38,15:45,7.0,15:48,10,1
74,Control,2,PrL-16,mCherry,9,OFF,17:53,18:08,15.0,18:17,24,1
75,Control,2,PrL-16,mCherry,10,ON,20:03,20:20,17.0,20:27,24,1


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [152]:
def create_pup_dataset(pup_images: int):
    START_PUP_CAPTURE_THRESH = 2
    END_PUP_CAPTURE_THRESH = 3

    min_trial = curr_mouse_df.head(1).Trial.item()
    max_trial = curr_mouse_df.tail(1).Trial.item()

    trials_done = []
    all_possible_trials = list(curr_mouse_df.Trial)
    images_recorded = 0

    while images_recorded < pup_images:

        trial = random.randint(min_trial, max_trial)
        trial_row = curr_mouse_df.loc[curr_mouse_df.Trial == trial]

        if len(trials_done) == len(curr_mouse_df):
            print("Insufficient contact group images...")
            break
        if (trial not in trials_done) and (trial in all_possible_trials):
            trial_row = curr_mouse_df[curr_mouse_df.Trial == trial]

            start_time = str(trial_row.Start.item())
            end_time = str(trial_row.Time_Latency_Contact_s.item())

            start_seconds = utils.time_to_sec(start_time) + START_PUP_CAPTURE_THRESH
            end_seconds = utils.time_to_sec(end_time) - END_PUP_CAPTURE_THRESH
            diff_seconds = end_seconds - start_seconds

            future_number_of_images = images_recorded + diff_seconds
            # Stop recording excess number of images.
            if future_number_of_images > pup_images:
                end_seconds = start_seconds + (pup_images - images_recorded)
                diff_seconds = end_seconds - start_seconds

            print(
                f"Starts at {start_seconds} seconds and end at {end_seconds} seconds of interval {diff_seconds} seconds"
            )
            create_dataset_pipeline(start_seconds, end_seconds, "child-mice", False)

            images_recorded += diff_seconds

            trials_done.append(trial)  # append unique trials
    split_train_test("child-mice", 0.8)

    train_size = len(os.listdir(f"{path_to_data}/train/child-mice"))
    test_size = len(os.listdir(f"{path_to_data}/test/child-mice"))

    print(
        f"Saved all {images_recorded} child image files and train has {train_size} pictures and test has {test_size} pictures!"
    )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [156]:
def create_final_parent_pup_dataset(images: int, create_parent: bool, create_child: bool):
    try:
        if create_child:
            utils.remove_all_files(f"{path_to_data}/train/child-mice")
            utils.remove_all_files(f"{path_to_data}/test/child-mice")
            create_pup_dataset(images)
        if create_parent:
            utils.remove_all_files(f"{path_to_data}/train/parent-mice")
            utils.remove_all_files(f"{path_to_data}/test/parent-mice")
            create_parent_dataset(images)
    except:
        var = traceback.format_exc()
        print(var)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [157]:
create_final_parent_pup_dataset(40, create_parent=False, create_child=True)

Starts at 547 seconds and end at 562 seconds of interval 15 seconds
Saved image as PrL-16-f53c4bb3-a1b6-4eb0-853d-bd7c1faf7240 at timestamp 09:07
Saved image as PrL-16-14b578cf-a09f-4bf3-ab37-a02ec0dcbd21 at timestamp 09:08
Saved image as PrL-16-617e9cc5-ea1b-497c-8841-11ab5b3b8173 at timestamp 09:09
Saved image as PrL-16-4504af9a-6f6b-4e7b-a34c-2957e0c0e606 at timestamp 09:10
Saved image as PrL-16-b0ddb07e-71ca-4779-9484-bfa677022e4a at timestamp 09:11
Saved image as PrL-16-672145d3-57e8-4ca6-b01b-e6362719ae0d at timestamp 09:12
Saved image as PrL-16-8eb32417-7108-4bd2-aae7-b77dc1b5d44b at timestamp 09:13
Saved image as PrL-16-16816d7a-b134-4cd4-9a95-67d26a1611b9 at timestamp 09:14
Saved image as PrL-16-25a656bb-06ab-4102-adac-0a4238125ac5 at timestamp 09:15
Saved image as PrL-16-4d2066ff-de63-439b-a0cd-b622e8ad9caa at timestamp 09:16
Saved image as PrL-16-9d16f9bd-fda4-4afa-9125-6c3d835c7342 at timestamp 09:17
Saved image as PrL-16-3eeeb4d9-4813-4a10-a5d5-08817dd5c484 at timestamp 09

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>